# Reference 
https://github.com/PeterZZQ/scMoMaT/blob/main/calc_pseudo_count/calc_pseudo_count.R

In [1]:
# rm(list = ls())
# gc()

library(Matrix)
library(BiocGenerics)
library(GenomicRanges)
library(IRanges)

#' Extend
#'
#' Resize GenomicRanges upstream and or downstream.
#' From \url{https://support.bioconductor.org/p/78652/}
#'
#' @param x A range
#' @param upstream Length to extend upstream
#' @param downstream Length to extend downstream
#' @param from.midpoint Count bases from region midpoint,
#' rather than the 5' or 3' end for upstream and downstream
#' respectively.
#'
#' @importFrom GenomicRanges trim
#' @importFrom BiocGenerics start strand end width
#' @importMethodsFrom GenomicRanges strand start end width
#' @importFrom IRanges ranges IRanges "ranges<-"
#' @export
#' @concept utilities
#' @return Returns a \code{\link[GenomicRanges]{GRanges}} object
#' @examples
#' Extend(x = blacklist_hg19, upstream = 100, downstream = 100)
Extend <- function(
    x,
    upstream = 0,
    downstream = 0,
    from.midpoint = FALSE
) {
    if (any(strand(x = x) == "*")) {
        warning("'*' ranges were treated as '+'")
    }
    on_plus <- strand(x = x) == "+" | strand(x = x) == "*"
    if (from.midpoint) {
        midpoints <- start(x = x) + (width(x = x) / 2)
        new_start <- midpoints - ifelse(
            test = on_plus, yes = upstream, no = downstream
        )
        new_end <- midpoints + ifelse(
            test = on_plus, yes = downstream, no = upstream
        )
    } else {
        new_start <- start(x = x) - ifelse(
            test = on_plus, yes = upstream, no = downstream
        )
        new_end <- end(x = x) + ifelse(
            test = on_plus, yes = downstream, no = upstream
        )
    }
    IRanges::ranges(x = x) <- IRanges::IRanges(start = new_start, end = new_end)
    x <- trim(x = x)
    return(x)
}

find_geneact <- function(peak.df, annotation.file, seq.levels, upstream = 2000, downstream = 0, verbose = FALSE,split="_"){
    # peak.df is the regions
    peak = peak.df
    # reformualte peak.df of the form "chromosome", "start", "end"
    peak.df <- do.call(what = rbind, args = strsplit(x = peak.df, split = split))
    peak.df <- as.data.frame(x = peak.df)
    colnames(x = peak.df) <- c("chromosome", "start", "end")
    
    # peak.df -> peaks.gr
    peaks.gr <- GenomicRanges::makeGRangesFromDataFrame(df = peak.df)
    BiocGenerics::start(peaks.gr[BiocGenerics::start(peaks.gr) == 0, ]) <- 1
    
    # gtf stores the annotation (reference genome)
    gtf <- rtracklayer::import(con = annotation.file)
    gtf <- GenomeInfoDb::keepSeqlevels(x = gtf, value = seq.levels, pruning.mode = "coarse")
    if (!any(GenomeInfoDb::seqlevelsStyle(x = gtf) == GenomeInfoDb::seqlevelsStyle(x = peaks.gr))) {
        GenomeInfoDb::seqlevelsStyle(gtf) <- GenomeInfoDb::seqlevelsStyle(peaks.gr)
    }
    # gtf.genes stores the genes 
    gtf.genes <- gtf[gtf$type == "gene"]
    
    # update the regions correspond to each gtf.genes, gtf.body_prom
    gtf.body_prom <- Extend(x = gtf.genes, upstream = upstream, downstream = downstream)
    
    # assign peaks.gr to nearest gene region
    gene.distances <- GenomicRanges::distanceToNearest(x = peaks.gr, subject = gtf.body_prom)
    # only leave the ones(regions) overlap with the gene regions(distance = 0)
    keep.overlaps <- gene.distances[rtracklayer::mcols(x = gene.distances)$distance == 
                                        0]
    peak.ids <- peaks.gr[S4Vectors::queryHits(x = keep.overlaps)]
    gene.ids <- gtf.genes[S4Vectors::subjectHits(x = keep.overlaps)]
    gene.ids$gene_name[is.na(gene.ids$gene_name)] <- gene.ids$gene_id[is.na(gene.ids$gene_name)]
    peak.ids$gene.name <- gene.ids$gene_name
    peak.ids <- as.data.frame(x = peak.ids)
    peak.ids$peak <- peak[S4Vectors::queryHits(x = keep.overlaps)]
    # new annotations should include peaks and corresponding gene.name
    annotations <- peak.ids[, c("peak", "gene.name")]
    
    return(annotations)
}


# # hyper-parameters
# species <- "Mouse"
# # upstream region size (base-pair)
# upstream <- 2000
# # downstream region size (base-pair)
# downstream <- 0

# path <- "./raw/"

# # regions chrX_start_end
# regions <- read.table(file = paste0(path, "regions.txt"), sep = ",", header = FALSE)[[1]]

# # location of reference genome
# if(species == "Mouse"){
#     A = find_geneact(peak.df = regions, annotation.file = "./reference_genome/Mus_musculus.GRCm38.84.gtf", 
#                      seq.levels = c(1:19, "X", "Y"), upstream = upstream, downstream = downstream, verbose = TRUE)
# } else if(species == "Human"){
#     A = find_geneact(peak.df = regions, annotation.file = "./reference_genome/Homo_sapiens.GRCh37.82.gtf", 
#                      seq.levels = c(1:22, "X", "Y"), upstream = upstream, downstream = downstream, verbose = TRUE)
# } else{
#     stop("species can only be Human or Mouse")
# }

# # output gene activity matrix
# write.table(A, file = paste0(path, "GxR.csv"), sep = ",")





Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Warning message:
“package ‘GenomicRanges’ was built under R version 4.1.2”
Loading required package: stats4

Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:Matrix’:

    expand, unname


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Warning message:
“package ‘IRanges’ was built unde

# PBMC 

In [13]:
pbmc_atac = SeuratDisk::LoadH5Seurat("dataset/multiome_pbmc_10k/pbmc_10x_atac_peak_counts.h5Seurat")

Validating h5Seurat file

Initializing ATAC with data

Adding counts for ATAC

Adding miscellaneous information for ATAC

Initializing RNA with data

Adding counts for RNA

Adding miscellaneous information for RNA

Initializing SCT with data

Adding counts for SCT

Adding scale.data for SCT

Adding variable feature information for SCT

Adding miscellaneous information for SCT

Adding reduction lsi

Adding cell embeddings for lsi

Adding feature loadings for lsi

Adding miscellaneous information for lsi

Adding reduction umap.atac

Adding cell embeddings for umap.atac

Adding miscellaneous information for umap.atac

Adding reduction wnn.umap

Adding cell embeddings for wnn.umap

Adding miscellaneous information for wnn.umap

Adding reduction pca

Adding cell embeddings for pca

Adding feature loadings for pca

Adding miscellaneous information for pca

Adding reduction umap.rna

Adding cell embeddings for umap.rna

Adding miscellaneous information for umap.rna

Adding graph wknn

Adding 

In [27]:
head(rownames(pbmc_atac))

[1] "chr1-10109-10357"   "chr1-180730-181630" "chr1-191491-191736"
[4] "chr1-267816-268196" "chr1-586028-586373" "chr1-629721-630172"

In [26]:
# hyper-parameters
#species <- "Human"
# upstream region size (base-pair)
upstream <- 2000
# downstream region size (base-pair)
downstream <- 0
regions = rownames(pbmc_atac)
A = find_geneact(peak.df = regions, annotation.file = "GRCg38_genes.gtf.gz", 
                 seq.levels = paste0('chr',c(1:22, "X", "Y")), upstream = upstream, downstream = downstream, 
                 verbose = TRUE,split="-")


In [32]:
head(A)
dim(A)

,peak,gene.name
,<chr>,<chr>
1,chr1-267816-268196,AP006222.2
2,chr1-586028-586373,AC114498.1
3,chr1-777634-779926,LINC01409
4,chr1-816881-817647,FAM87B
5,chr1-819912-823500,LINC01128
6,chr1-825827-825889,LINC01128


[1] 79559     2

In [34]:
write.table(A, file = paste0('dataset/multiome_pbmc_10k/pbmc_peak_GxR.csv'), sep = ",")

In [ ]:
df <- gather(cbind(A,1), 'Var2', 'Value' 2:4)

In [31]:
graph_from_dataframe <- igraph::graph.data.frame(A)
adjacency_matrix <- igraph::get.adjacency(graph_from_dataframe, sparse = TRUE)

In [33]:
dim(adjacency_matrix)

ERROR: Error in eval(expr, envir, enclos): object 'adjacency_matrix' not found


In [ ]:
if (!any(GenomeInfoDb::seqlevelsStyle(x = gtf) == GenomeInfoDb::seqlevelsStyle(x = peaks.gr))) {
    GenomeInfoDb::seqlevelsStyle(gtf) <- GenomeInfoDb::seqlevelsStyle(peaks.gr)
}

In [16]:
head(regions)

[1] "chr1-10109-10357"   "chr1-180730-181630" "chr1-191491-191736"
[4] "chr1-267816-268196" "chr1-586028-586373" "chr1-629721-630172"

# BMMC

In [2]:
bmmc_atac = SeuratDisk::LoadH5Seurat("dataset/bmmc/multiome_bmmc_site1_or_donor1_ATAC_pmat.h5seurat")

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

Validating h5Seurat file

Initializing ATAC with data

Adding counts for ATAC

Adding miscellaneous information for ATAC

Loading required package: Signac


Attaching package: ‘Signac’


The following object is masked _by_ ‘.GlobalEnv’:

    Extend


Initializing RNA with data

Adding counts for RNA

Adding miscellaneous information for RNA

Adding command information

Adding cell-level metadata

Adding miscellaneous information

Adding tool-specific results



In [7]:
head(rownames(bmmc_atac))
length(rownames(bmmc_atac))

[1] "chr1-9776-10668"    "chr1-180726-181005" "chr1-181117-181803"
[4] "chr1-191133-192055" "chr1-267562-268456" "chr1-629497-630394"

[1] 116490

In [4]:
# hyper-parameters
#species <- "Human"
# upstream region size (base-pair)
upstream <- 2000
# downstream region size (base-pair)
downstream <- 0
regions = rownames(bmmc_atac)
A = find_geneact(peak.df = regions, annotation.file = "GRCg38_genes.gtf.gz", 
                 seq.levels = paste0('chr',c(1:22, "X", "Y")), upstream = upstream, downstream = downstream, 
                 verbose = TRUE,split="-")
write.table(A, file = paste0('dataset/bmmc/bmmc_peak_GxR.csv'), sep = ",")

# HPAP

In [11]:
hpap_atac = SeuratDisk::LoadH5Seurat("/project/mingyaolpc/myylee/scmint/methods_eval/dataset/hpap/multiome/hpap_multiomeATAC_4donors.h5seurat")



Validating h5Seurat file

Initializing ATAC with data

Adding counts for ATAC

Adding miscellaneous information for ATAC

Adding command information

Adding cell-level metadata

Adding miscellaneous information

Adding tool-specific results



In [12]:
head(rownames(hpap_atac))
length(rownames(hpap_atac))

[1] "chr1-10064-10263"   "chr1-180628-180978" "chr1-181263-181599"
[4] "chr1-191296-191607" "chr1-267866-268080" "chr1-629134-629361"

[1] 172871

In [13]:
# hyper-parameters
#species <- "Human"
# upstream region size (base-pair)
upstream <- 2000
# downstream region size (base-pair)
downstream <- 0
regions = rownames(hpap_atac)
A = find_geneact(peak.df = regions, annotation.file = "GRCg38_genes.gtf.gz", 
                 seq.levels = paste0('chr',c(1:22, "X", "Y")), upstream = upstream, downstream = downstream, 
                 verbose = TRUE,split="-")
write.table(A, file = paste0('/project/mingyaolpc/myylee/scmint/methods_eval/dataset/hpap/hpap_peak_GxR.csv'), sep = ",")



# Mouse skin share-seq

In [2]:
mouse_skin_atac = SeuratDisk::LoadH5Seurat("/project/mingyaolpc/myylee/scmint/methods_eval/dataset/mouse_skin/mouse_skin_shareseq_atac.h5seurat")



Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

Validating h5Seurat file

Initializing ATAC with data

Adding counts for ATAC

Adding miscellaneous information for ATAC

Adding command information

Adding cell-level metadata

Adding miscellaneous information

Adding tool-specific results



In [3]:
head(rownames(mouse_skin_atac))
length(rownames(mouse_skin_atac))

[1] "chrX-143482906-143483206" "chr6-3200976-3201276"    
[3] "chr9-123461850-123462150" "chr1-56782095-56782395"  
[5] "chr9-56223668-56223968"   "chr1-88277459-88277759"

[1] 344592

In [4]:
table(gsub("-.*","",rownames(mouse_skin_atac)))


 chr1 chr10 chr11 chr12 chr13 chr14 chr15 chr16 chr17 chr18 chr19  chr2  chr3 
23891 18717 23570 14801 16622 13526 15166 12972 14076 11862 10164 26635 18402 
 chr4  chr5  chr6  chr7  chr8  chr9  chrX 
22227 20489 18856 19908 17395 19025  6288 

In [19]:
# hyper-parameters
#species <- "Mouse"
# upstream region size (base-pair)
upstream <- 2000
# downstream region size (base-pair)
downstream <- 0
regions = rownames(mouse_skin_atac)
A = find_geneact(peak.df = regions, annotation.file = "mm10_genes.gtf.gz", 
                 seq.levels = paste0('chr',c(1:19, "X", "Y")), upstream = upstream, downstream = downstream, 
                 verbose = TRUE,split="-")
write.table(A, file = paste0('/project/mingyaolpc/myylee/scmint/methods_eval/dataset/mouse_skin/mouse_skin_peak_GxR.csv'), sep = ",")



In [18]:
tail(gtf$gene_name,n=100)

[1] "AC163611.2" "AC163611.2" "AC163611.2" "AC163611.2" "AC163611.2"
  [6] "AC163611.2" "AC140365.1" "AC140365.1" "AC140365.1" "AC140365.1"
 [11] "AC140365.1" "AC140365.1" "AC140365.1" "AC140365.1" "AC140365.1"
 [16] "AC140365.1" "AC140365.1" "AC140365.1" "AC124606.2" "AC124606.2"
 [21] "AC124606.2" "AC124606.2" "AC124606.2" "AC124606.2" "AC124606.2"
 [26] "AC124606.2" "AC124606.2" "AC124606.2" "AC124606.2" "AC124606.1"
 [31] "AC124606.1" "AC124606.1" "AC124606.1" "AC124606.1" "AC124606.1"
 [36] "AC124606.1" "AC124606.1" "AC124606.1" "AC124606.1" "AC124606.1"
 [41] "AC133095.2" "AC133095.2" "AC133095.2" "AC133095.2" "AC133095.2"
 [46] "AC133095.2" "AC133095.2" "AC133095.2" "AC133095.2" "AC133095.2"
 [51] "AC133095.2" "AC133095.1" "AC133095.1" "AC133095.1" "AC133095.1"
 [56] "AC133095.1" "AC133095.1" "AC133095.1" "AC133095.1" "AC133095.1"
 [61] "AC133095.1" "AC133095.1" "AC234645.1" "AC234645.1" "AC234645.1"
 [66] "AC234645.1" "AC234645.1" "AC149090.1" "AC149090.1" "AC149090.1"
 [71] "AC149090.1" "AC149090.1" "AC149090.1" "AC149090.1" "AC149090.1"
 [76] "AC149090.1" "AC149090.1" "AC149090.1" "AC149090.1" "AC149090.1"
 [81] "AC149090.1" "AC149090.1" "AC149090.1" "AC149090.1" "AC149090.1"
 [86] "AC149090.1" "AC149090.1" "AC149090.1" "AC149090.1" "AC149090.1"
 [91] "AC149090.1" "AC149090.1" "AC149090.1" "AC149090.1" "AC149090.1"
 [96] "AC149090.1" "AC149090.1" "AC149090.1" "AC149090.1" "AC149090.1"